In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Завантаження даних
df = pd.read_csv('Housing.csv')

# Вибір ознак (Features) та цільової змінної (Target)
features = ['area', 'bathrooms', 'bedrooms']
target = 'price'

X_original = df[features].values
y = df[target].values.reshape(-1, 1) # Робимо вектор-стовпчик

# Нормалізація даних (Standard Scaler: z = (x - u) / s)
# Це критично важливо для градієнтного спуску
mean = np.mean(X_original, axis=0)
std = np.std(X_original, axis=0)
X_normalized = (X_original - mean) / std

# Додаємо стовпчик одиниць для вільного члена (w0 - bias)
m = len(y)
X = np.hstack((np.ones((m, 1)), X_normalized))

print(f"Розмірність матриці X: {X.shape}")
print(f"Розмірність вектора y: {y.shape}")

Розмірність матриці X: (545, 4)
Розмірність вектора y: (545, 1)


In [2]:
# Гіпотеза
def hypothesis(X, w):
    return np.dot(X, w)


In [3]:
# Функція втрат
def compute_cost(X, y, w):
    m = len(y)
    predictions = hypothesis(X, w)
    cost = (1 / (2 * m)) * np.sum((predictions - y) ** 2)
    return cost

In [4]:
# Градієнтний спуск
def gradient_descent_step(X, y, w, learning_rate):
    m = len(y)
    predictions = hypothesis(X, w)
    error = predictions - y
    
    # Обчислення градієнта: (1/m) * X.T * error
    gradient = (1 / m) * np.dot(X.T, error)
    
    # Оновлення ваг
    w_new = w - learning_rate * gradient
    return w_new

In [6]:
# Найкращі параметри w для датасету (Цикл навчання)

w_gd = np.zeros((X.shape[1], 1)) # Початкові ваги - нулі
learning_rate = 0.01
iterations = 2000

cost_history = []
for i in range(iterations):
    w_gd = gradient_descent_step(X, y, w_gd, learning_rate)
    cost = compute_cost(X, y, w_gd)
    cost_history.append(cost)

print("Знайдені параметри (Градієнтний спуск):")
print(w_gd)
print(f"Фінальна вартість (Loss): {cost_history[-1]}")

Знайдені параметри (Градієнтний спуск):
[[4766729.2388224 ]
 [ 821214.21655634]
 [ 695807.93135737]
 [ 299984.13076566]]
Фінальна вартість (Loss): 895585024988.8679


In [7]:
# Знайдіть ці ж параметри за допомогою аналітичного рішення
def analytical_solution(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

w_analytical = analytical_solution(X, y)
cost_analytical = compute_cost(X, y, w_analytical)

print("Знайдені параметри (Аналітичне рішення):")
print(w_analytical)
print(f"Фінальна вартість (Loss): {cost_analytical}")

Знайдені параметри (Аналітичне рішення):
[[4766729.24770642]
 [ 821214.14349519]
 [ 695808.52272537]
 [ 299983.57107963]]
Фінальна вартість (Loss): 895585024988.6597


In [9]:
# Створення та навчання моделі
# Ми подаємо X_normalized (без одиниць), бо sklearn додасть bias сам
sklearn_model = LinearRegression()
sklearn_model.fit(X_normalized, y)

# Отримання параметрів
w0_sklearn = sklearn_model.intercept_[0]
w_rest_sklearn = sklearn_model.coef_.flatten()

# Збираємо все в один вектор для порівняння
w_sklearn = np.hstack(([w0_sklearn], w_rest_sklearn)).reshape(-1, 1)

print("\n=== ПОРІВНЯЛЬНА ТАБЛИЦЯ ===")
comparison = pd.DataFrame({
    'Gradient Descent': w_gd.flatten(),
    'Analytical': w_analytical.flatten(),
    'Scikit-Learn': w_sklearn.flatten()
}, index=['Bias (Intercept)', 'Area', 'Bathrooms', 'Bedrooms'])

print(comparison)

# Перевірка на близькість значень
if np.allclose(w_gd, w_analytical, rtol=0.01) and np.allclose(w_analytical, w_sklearn, rtol=0.01):
    print("\n Всі методи дали однакові результати.")
else:
    print("\n Є розбіжності.")


=== ПОРІВНЯЛЬНА ТАБЛИЦЯ ===
                  Gradient Descent    Analytical  Scikit-Learn
Bias (Intercept)      4.766729e+06  4.766729e+06  4.766729e+06
Area                  8.212142e+05  8.212141e+05  8.212141e+05
Bathrooms             6.958079e+05  6.958085e+05  6.958085e+05
Bedrooms              2.999841e+05  2.999836e+05  2.999836e+05

 Всі методи дали однакові результати.
